In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from quapopt.data_analysis.data_handling import (CoefficientsType,
                                                 CoefficientsDistribution,
                                                 CoefficientsDistributionSpecifier,
                                                 HamiltonianModels)
from quapopt.hamiltonians.generators import build_hamiltonian_generator
from quapopt.optimization.QAOA.simulation.QAOARunnerExpValues import QAOARunnerExpValues
from quapopt.optimization.parameter_setting import (OptimizerType)
from quapopt.optimization.parameter_setting.variational.QAOAOptimizationRunner import QAOAOptimizationRunner
from quapopt.optimization.parameter_setting.variational.scipy_tools.ScipyOptimizerWrapped import ScipyOptimizerWrapped
from quapopt.optimization.parameter_setting.non_adaptive_optimization.SimpleGridOptimizer import SimpleGridOptimizer
from quapopt.optimization.parameter_setting import (ParametersBoundType as PBT)

In [3]:
coefficients_type = CoefficientsType.DISCRETE
coefficients_distribution = CoefficientsDistribution.Uniform
coefficients_distribution_properties = {'low': -1, 'high': 1, 'step': 1}
coefficients_distribution_specifier = CoefficientsDistributionSpecifier(CoefficientsType=coefficients_type,
                                                                        CoefficientsDistributionName=coefficients_distribution,
                                                                        CoefficientsDistributionProperties=coefficients_distribution_properties)

# We generate a Hamiltonian instance. In this case it's a random Sherrington-Kirkpatrick Hamiltonian
hamiltonian_model = HamiltonianModels.SherringtonKirkpatrick
localities = (1, 2,)

generator_cost_hamiltonian = build_hamiltonian_generator(hamiltonian_model=hamiltonian_model,
                                                         localities=localities,
                                                         coefficients_distribution_specifier=coefficients_distribution_specifier)





## Large-scale p=1 QAOA simulation
* Here we will test simulation of p=1 2-local expected values on large number of qubits.
* NOTE: this is optimized for GPUs, so if you don't have CUDA, it might be slow for more than 100 qubits
* To make it more interesting, we will use analytical expressions for optimal betas for each gamma, so we only need to optimize over single parameter. (note that this works only in simulation)

In [4]:
number_of_qubits = 100
seed_cost_hamiltonian = 1

cost_hamiltonian = generator_cost_hamiltonian.generate_instance(number_of_qubits=number_of_qubits,
                                                                seed=seed_cost_hamiltonian,
                                                                read_from_drive_if_present=True,

                                                                )
cost_hamiltonian.solve_hamiltonian()


print("Ground state energy:",cost_hamiltonian.lowest_energy)

File not found!
FILE NOT FOUND!


/home/fipeczek/python_packages/quapopt_versions/current_public/quantum-approximate-optimization/quapopt/hamiltonians/representation/problem_formulations.py:29: SyntaxWarning: invalid escape sequence '\s'
  \sum_{i<=j} J_{ij} (1-2*x_i)*(1-2*xj) + \sum_i h_i (1-2xi)


Ground state energy: -748.0


In [5]:
number_of_function_calls = 100

single_bound = (0, np.pi)

#let's use grid search optimizer
#just single parameter
classical_optimizer = SimpleGridOptimizer(parameter_bounds=[(PBT.RANGE, single_bound)],
                                            max_trials=number_of_function_calls)

#uncomment to use COBYLA optimizer instead
# classical_optimizer = ScipyOptimizerWrapped(parameters_bounds=[single_bound],
#                                             argument_names = ['Angles-0'],
                                            # optimizer_name='COBYLA',
                                            # optimizer_kwargs=None,
                                            # basinhopping=True,
                                            # basinhopping_kwargs={'niter': 3},
                                            # starting_point=[0.05]
                                            # )

In [6]:
#if set to None, the simulator will be chosen automatically
simulator = None
precision = np.float32

qaoa_runner_analytical = QAOARunnerExpValues(hamiltonian_representations_cost=[cost_hamiltonian],
                                             store_full_information_in_history=True,
                                             simulator_name=simulator,
                                             precision_float=precision)
qaoa_optimizer_analytical = QAOAOptimizationRunner(qaoa_runner_analytical)

#whether to store all 2-local expected values in memory
store_correlators = True

#whether to optimize over betas or use analytical betas; if False, you also need to adjust optimizer setting in previous cell to include two parameters instead of one
analytical_betas = True
best_result_analytical, optimization_res_analytical = qaoa_optimizer_analytical.run_optimization(qaoa_depth=1,
                                                                                                 number_of_function_calls=number_of_function_calls,
                                                                                                 classical_optimizer=classical_optimizer,
                                                                                                 measurement_noise=None,
                                                                                                 show_progress_bar=True,
                                                                                                 verbosity=0,
                                                                                                 analytical_betas=analytical_betas,
                                                                                                 store_correlators=store_correlators)
print(best_result_analytical)


CUDA simulator is not available. Using cython instead.


  0%|          | 0/100 [00:00<?, ?it/s]

[(np.float32(-302.99402), (None, <quapopt.optimization.QAOA.QAOAResult object at 0x7c3d655dfd10>))]


In [7]:
print(best_result_analytical[0])
#accessing correlators
#Note: THOSE ARE REALLY WEIGHTED CORRELATORS (LOCAL EXPECTED VALUES), i.e., J_{ij} * Z_iZ_j;
#To get ZiZj, please divide by coefficients
best_correlators =  best_result_analytical[0][1][1].correlators
best_energy = best_result_analytical[0][0]


print('best energy:', best_energy)
print(np.round(best_result_analytical[0][1][1].correlators,3))

(np.float32(-302.99402), (None, <quapopt.optimization.QAOA.QAOAResult object at 0x7c3d655dfd10>))
best energy: -302.99402
[[-0.043 -0.055 -0.029 ... -0.04  -0.077 -0.074]
 [ 0.    -0.043 -0.055 ... -0.037 -0.063 -0.044]
 [ 0.     0.    -0.043 ... -0.055 -0.074 -0.07 ]
 ...
 [ 0.     0.     0.    ... -0.043 -0.089 -0.07 ]
 [ 0.     0.     0.    ...  0.    -0.043 -0.063]
 [ 0.     0.     0.    ...  0.     0.    -0.043]]


### Quantum Relax and Round (QRR) algorithm

We can also apply QRR algorithm to obtain candidate solutions from exp-values of the cost Hamiltonian. The QRR algorithm is based on a matrix of 2-local correlations. The algorithm is described in Ref [1].

[1] Dupont, Maxime, and Bhuvanesh Sundar. "Extending relax-and-round combinatorial optimization solvers with quantum correlations." Physical Review A 109, no. 1 (2024): 012429.

In [8]:
if store_correlators:
    #This is possible only if we stored correlators
    best_result_qrr, (
    opt_res_qrr, best_bitstrings_qrr, _) = qaoa_optimizer_analytical.apply_QRR_to_optimization_results(
        return_full_history=False,
        show_progress_bar=True)


  0%|          | 0/100 [00:00<?, ?it/s]

In [9]:
xs_qrr = [x[0] for x in opt_res_qrr.trials_dataframe['Angles'].values]
ys_qrr = opt_res_qrr.trials_dataframe['Energy']

### Visualization

* Let's take a look at a simple visualization of the NDAR optimization.


In [10]:
import plotly
import plotly.graph_objects as go
plotly.io.templates.default = "plotly"
plotly.offline.init_notebook_mode(connected=True)

In [11]:
xs = optimization_res_analytical.trials_dataframe['ARG-0']
ys = optimization_res_analytical.trials_dataframe['FunctionValue']

fig = go.Figure()
fig.add_trace(go.Scatter(x=xs, y=ys, mode='markers', name='QAOA p=1'))
if store_correlators:
    fig.add_trace(go.Scatter(x=xs_qrr, y=ys_qrr, mode='markers', name='QAOA p=1 + QRR'))

fig.update_layout(
    title=f"QAOA p=1 with analytical betas performance for {number_of_qubits} qubits on {cost_hamiltonian.hamiltonian_class_specifier.get_description_string()}",
    xaxis_title="\gamma",
    yaxis_title="Energy",
    template="plotly",
    font=dict(size=14),
    hovermode="x unified")

plotly.offline.plot(fig,
                    filename=f'../temp/p1_simulator_analytical_betas_n={number_of_qubits}.html')
fig.show()

<>:11: SyntaxWarning:

invalid escape sequence '\g'

<>:11: SyntaxWarning:

invalid escape sequence '\g'

/tmp/ipykernel_74373/3280062010.py:11: SyntaxWarning:

invalid escape sequence '\g'

